In [111]:
import numpy as np
import pandas as pd

In [112]:
dataset = pd.read_csv("../data/training_data.csv")
dataset = pd.DataFrame(dataset)

In [113]:
dataset

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,619,CASH_IN,386385.08,C421351828,4669568.85,5055953.92,C1977099364,506035.06,119649.98,0.0,0.0
1,164,CASH_IN,212458.78,C83569848,234635.00,447093.78,C1690589535,806037.88,593579.10,0.0,0.0
2,382,PAYMENT,19967.60,C852995095,3634.00,0.00,M1695416333,0.00,0.00,0.0,0.0
3,180,CASH_OUT,527616.51,C61761046,180216.00,0.00,C577654587,92157.10,619773.61,0.0,0.0
4,36,TRANSFER,206067.85,C758004147,0.00,0.00,C2143015292,2131494.48,2337562.32,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
143921,236,CASH_IN,184782.46,C2000187312,1512205.59,1696988.05,C1303211682,293982.59,109200.13,0.0,0.0
143922,347,PAYMENT,31391.33,C1729609363,31152.00,0.00,M602706381,0.00,0.00,0.0,0.0
143923,311,CASH_IN,255521.98,C1034541925,1237508.67,1493030.65,C1088098741,768396.97,512874.99,0.0,0.0
143924,11,CASH_OUT,78284.35,C1847843274,31562.00,0.00,C697452681,2324888.80,2420233.51,0.0,0.0


In [114]:
dataset.columns = dataset.columns.str.strip()  # Removes leading and trailing spaces
print(dataset.isnull().sum()) # Check for missing values
print("---")
print(dataset.duplicated().sum()) # Check for duplicate values


step              0
type              0
amount            0
nameOrig          1
oldbalanceOrg     1
newbalanceOrig    1
nameDest          1
oldbalanceDest    1
newbalanceDest    1
isFraud           1
isFlaggedFraud    1
dtype: int64
---
0


In [115]:
dataset.dropna(inplace=True)
dataset.drop_duplicates(inplace=True)

In [116]:
'''
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming your dataset is already loaded, e.g.:
# dataset = pd.read_csv("your_file.csv")

# Select independent features and dependent target
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features
Y = dataset['isFraud'] # Target

# Create a new DataFrame that includes both X and Y for plotting
data_with_target = X.copy()  # Copy X to avoid modifying it directly
data_with_target['isFraud'] = Y  # Add the target 'isFraud' to the DataFrame

# Plot pairwise scatter plots for each feature in X against Y
for col in X.columns:
    plt.figure(figsize=(6, 4))
    sns.scatterplot(x=col, y='isFraud', data=data_with_target)
    plt.title(f'Scatter plot of {col} vs isFraud')
    plt.show()
'''

'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\n# Assuming your dataset is already loaded, e.g.:\n# dataset = pd.read_csv("your_file.csv")\n\n# Select independent features and dependent target\nX = dataset[[\'amount\',\'oldbalanceOrg\',\'newbalanceOrig\',\'oldbalanceDest\',\'newbalanceDest\']] # Features\nY = dataset[\'isFraud\'] # Target\n\n# Create a new DataFrame that includes both X and Y for plotting\ndata_with_target = X.copy()  # Copy X to avoid modifying it directly\ndata_with_target[\'isFraud\'] = Y  # Add the target \'isFraud\' to the DataFrame\n\n# Plot pairwise scatter plots for each feature in X against Y\nfor col in X.columns:\n    plt.figure(figsize=(6, 4))\n    sns.scatterplot(x=col, y=\'isFraud\', data=data_with_target)\n    plt.title(f\'Scatter plot of {col} vs isFraud\')\n    plt.show()\n'

In [164]:
X = dataset[['amount','oldbalanceOrg','newbalanceOrig','oldbalanceDest','newbalanceDest']] # Features (independent)
Y = dataset['isFraud'].values.reshape(-1, 1) # Target (dependent)
X = pd.DataFrame(X)
Y = pd.DataFrame(Y)

In [172]:
target_dict = Y.value_counts().to_dict()
n_labels = len(target_dict)
print(np.unique(Y))
print(target_dict)
print(n_labels)
print(type(Y)==pd.DataFrame)

[0. 1.]
{(0.0,): 143733, (1.0,): 192}
2
True


In [136]:
print(X.shape)
print(Y.shape)

(143925, 5)
(143925, 1)


In [119]:
X.iloc[1:3]


,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
1,212458.78,234635.0,447093.78,806037.88,593579.1
2,19967.60,3634.0,0.00,0.00,0.0


In [120]:
def split(rows, qt):
    true_row=[]; false_row=[]

    for i in rows:
        if qt.match(i):
            true_row.append(i)
        else:
            false_row.append(i)
    return true_row,false_row


In [121]:
class qt:
    def _init_(self,col,val):
        self.col = col
        self.val = val
    
    def match(self, ex):
        val = ex[self.col]
        if is_numeric(val):
            return val>= self.val
        else:
            return val == self.val
    
    def __repr__(self):
        condition = "=="
        if is_numeric(self.val):
            condition = ">="
        return "Is %s %s %s ?" % (header[self.col], condition, str(self.val))


In [134]:
# Nodes
class node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = None

    def is_leaf(self):
        if self.value != None:
            return self.value

In [132]:
# Decision Tree
class decision_tree:
    def __init__(self, min_samples=5, max_depth=100, n_features=None):
        self.min_samples = min_samples
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self,x,y):
        if not self.n_features: # To ensure no. of features(columns) <= no. of actual features
            self.n_features = x.shape[1]
        else:
            self.n_features = min(x.shape[1],self.n_features)
        self.root = self.build_tree(x,y)



    depth = 0 # Initialize depth of tree
    def build_tree(self,x,y,depth):
        # Check if stopping criteria reached
        # If not find the best split
        # Create child nodes based on split condition

        n_samples = x.shape[0] # No. of sample points = no. of rows
        n_features = x.shape[1] # No. of features = no. of columns
        n_labels = len(np.unique(y)) # No. of unique labels

        if depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples: # Stopping criteria
            leaf_label = self.majority_label(y) # Label of leaf should be the majority label in the node
            return node(value=leaf_label) # Creates a leaf node and exits build_tree function

    def majority_label(self,y):
        if type(y) != pd.DataFrame:
            y = pd.DataFrame(y)
        y_dict = y.value_counts().to_dict()

        max_key,max_val = "",0
        for k,v in y_dict.items():
            if v>=max_val:
                max_key,max_val=k,v
        return max_key
        


        


In [153]:
# Making dictionary of each value in column and how many times it occurs
dict = dataset['type'].value_counts().to_dict()
for k,v in dict.items():
    print(k,"-->",v)

len(dict)

CASH_OUT --> 50733
PAYMENT --> 48683
CASH_IN --> 31397
TRANSFER --> 12154
DEBIT --> 958


5

In [123]:
# Subsetting dataset
dict = dataset['isFraud'].value_counts().to_dict()
for k,v in dict.items():
    print(k,"-->",v)

subset= dataset[dataset['isFraud']==1.0]
subset.head()

0.0 --> 143733
1.0 --> 192


,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
550,61,TRANSFER,1735466.52,C182416415,1735466.52,0.00,C180492809,0.00,0.00,1.0,0.0
636,558,TRANSFER,316877.85,C350145239,316877.85,0.00,C1384673123,0.00,0.00,1.0,0.0
864,142,CASH_OUT,78810.30,C688380113,78810.30,0.00,C1780087922,802788.52,881598.82,1.0,0.0
870,275,CASH_OUT,513082.74,C1294288160,513082.74,0.00,C1140210219,32452.79,545535.53,1.0,0.0
1472,617,TRANSFER,2542664.27,C786455622,2542664.27,2542664.27,C661958277,0.00,0.00,1.0,1.0


In [124]:
'''
def entropy(col,dataset):
    subset_fraud = dataset[dataset['isFraud']==1.0] # Taking only the rows with isFraud=1
    col_len = len(subset_fraud) # Length of the column
    dict_fraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset 

    ent=0 # Initialize entropy

    for k,v in dict_fraud.items():
        p = v/col_len # Probability of occurence of value in entire column
        ent -= p* np.log2(p)

    subset_nfraud = dataset[dataset['isFraud']==0.0] # Taking only the rows with isFraud=0
    col_len = len(subset_nfraud) # Length of the column
    dict_nfraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset 

    for k,v in dict_nfraud.items():
        p = v/col_len # Probability of occurence of value in entire column
        ent -= p* np.log2(p)


    return ent
    
entropy('isFraud',dataset)
'''

"\ndef entropy(col,dataset):\n    subset_fraud = dataset[dataset['isFraud']==1.0] # Taking only the rows with isFraud=1\n    col_len = len(subset_fraud) # Length of the column\n    dict_fraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset \n\n    ent=0 # Initialize entropy\n\n    for k,v in dict_fraud.items():\n        p = v/col_len # Probability of occurence of value in entire column\n        ent -= p* np.log2(p)\n\n    subset_nfraud = dataset[dataset['isFraud']==0.0] # Taking only the rows with isFraud=0\n    col_len = len(subset_nfraud) # Length of the column\n    dict_nfraud = subset_fraud[col].value_counts().to_dict() # Making dict of each unique val and no. of occurences in subset \n\n    for k,v in dict_nfraud.items():\n        p = v/col_len # Probability of occurence of value in entire column\n        ent -= p* np.log2(p)\n\n\n    return ent\n    \nentropy('isFraud',dataset)\n"

In [125]:
'''Entropy Calculation for a column

def entropy_column(col,target):
    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences
    ent = 0
    for k,v in att_dict.items(): # For each unique value
        subset = dataset[dataset[col]==k] # Make subset based on unique attribute k
        length = v # Length of subset = no. of occureneces

        target_dict = subset[target].value_counts().to_dict()
        for i,j in target_dict.items():
            p = j/length # Occurences of 1s/0s divided by len of column
            ent -= p * np.log2(p)

    return ent

entropy_column('isFraud','isFraud')       

'''

"Entropy Calculation for a column\n\ndef entropy_column(col,target):\n    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences\n    ent = 0\n    for k,v in att_dict.items(): # For each unique value\n        subset = dataset[dataset[col]==k] # Make subset based on unique attribute k\n        length = v # Length of subset = no. of occureneces\n\n        target_dict = subset[target].value_counts().to_dict()\n        for i,j in target_dict.items():\n            p = j/length # Occurences of 1s/0s divided by len of column\n            ent -= p * np.log2(p)\n\n    return ent\n\nentropy_column('isFraud','isFraud')       \n\n"

In [126]:
# Entropy calculation for an attribute in a column
def entropy_attribute(col,val,target):
    subset = dataset[dataset[col]==val] # Make subset based on attribute 
    
    ent = 0 # Initialize entropy value

    target_dict = subset[target].value_counts().to_dict()
    for k,v in target_dict.items():
        p = v/len(subset) # Occurences of 1s/0s divided by len of column
        ent -= p * np.log2(p) # Entropy of attribute

    return ent

entropy_attribute('isFraud',1.0,'isFraud')

np.float64(0.0)

In [127]:
# Entropy of entire dataset 
def entropy_dataset(target):
    target_dict = dataset[target].value_counts().to_dict()
    ent = 0
    for k,v in target_dict.items():
        p = v/len(dataset)
        ent -= p * np.log2(p)
    
    return ent

entropy_dataset('isFraud')

np.float64(0.014663273789461197)

In [131]:
# Information Gain Calculation
def information_gain(col,target):
    len_data = len(dataset) # Length of dataset
    att_dict = dataset[col].value_counts().to_dict() # Make dictionary of unique attribute and no. of occurences

    ent_data = entropy_dataset(target) # Calculate entropy of dataset
    ent_att = 0 # Initalize entropy of attributes of a column

    for k,v in att_dict.items():
        p = v / len_data # No. of occurences of attributes / length of dataset
        ent_att -= p * entropy_attribute(col,k,target) # Sum of entropy of all attributes in a column
    
    info_gain = ent_data + ent_att # Calculate information gain

    return info_gain

information_gain('type','isFraud')
        

np.float64(0.0020657492205213446)

In [ ]:
# Gini Index Calculation
